# Voxel VAE-GAN Training

This notebook is designed to provide a wholistic vae-gan training experience. You can adjust the model and training parameters through the sacred configuration file, you can view training progress in tensorboard, and you can (wip) create reconstructions with the saved models!

References:

* https://github.com/anitan0925/vaegan/blob/master/examples/train.py
  * Runs 20 epochs on separate VAE and GAN then 200 on VAEGAN
* https://github.com/jlindsey15/VAEGAN/blob/master/main.py
  * Almost clear code for vaegan paper
* https://arxiv.org/pdf/1512.09300.pdf
  * vaegan paper
* https://github.com/timsainb/Tensorflow-MultiGPU-VAE-GAN
  * Best code yet!
  

## Setup

In [1]:
import env
from train_vaegan import train_vaegan
from data.thingi10k import Thingi10k
from data.modelnet10 import ModelNet10
from data import MODELNET10_TOILET_INDEX, MODELNET10_SOFA_INDEX, MODELNET10_SOFA_TOILET_INDEX
from models import MODEL_DIR


# plot things
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2

## Prepare Sacred Experiment

In [2]:
from sacred.observers import FileStorageObserver
from sacred import Experiment
import os

ex = Experiment(name='voxel_vaegan_notebook', interactive=True)
ex.observers.append(FileStorageObserver.create('experiments_vaegan'))

@ex.main
def run_experiment(cfg):
    train_vaegan(cfg)

import datetime
last_model_dir = None

## Prepare Model Config

The model dir is generated with a timestamp. This keeps you from overwriting past results and keeps results separate to avoid confusing tensorboard.

But be warned! These model dirs can take up space, so you might need to periodically go back and delete ones you do not care about.

Also, if you ever train a model that you would really like to keep, I recommend moving it to a new directory with a special name like "best_model_ever".

In [3]:
DATASET_CLASS = 'ModelNet10'
#INDEX = MODELNET10_SOFA_TOILET_INDEX
#INDEX = MODELNET10_SOFA_INDEX
INDEX = MODELNET10_TOILET_INDEX

def make_cfg():
    model_dir = os.path.join(
        MODEL_DIR,
        'voxel_vaegan1/modelnet10/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')))
    print(model_dir)
    os.makedirs(model_dir)

    cfg = {
        'cfg': {
            "dataset": {
                "class": DATASET_CLASS,
                "index": INDEX,
                #"tag": "animal",
                #"filter_id": 126660,
                #"pctile": 1.0,
                "splits": True
                #"splits": {
                #    "train": .8,
                #    "dev": .1,
                #    "test": .1
                #}
            },
            "generator": {
                "verbose": True,
                "pad": True
            }, 
            "model": {
                "ckpt_dir": model_dir,
                "voxels_dim": 32,
                "batch_size": 128,
                # Do 0.0001 for 1 epoch, then 0.001 for rest of training
                #"learning_rate": [(1, 0.0001), (None, 0.001)],
                #"learning_rate": 0.0001,
                "enc_lr": 0.0001,  # 0.001 hits nan loss eventually
                "dec_lr": 0.0001,
                "dis_lr": 0.0001,
                "epochs": 201,
                "keep_prob": 0.8,
                "kl_div_loss_weight": 100,
                "recon_loss_weight": 10000,
                "ll_weight": 1,
                "dec_weight": 100,
                "latent_dim": 100,
                "verbose": True,
                "debug": True,
                "input_repeats": 1,
                "display_step": 1,
                #"example_stl_id": 126660,
                "voxel_prob_threshold": 0.065,
                "dev_step": 5,
                "save_step": 5,
                'launch_tensorboard': True,
                'tb_dir': 'tb',
                #'tb_compare': [('best_sofa_and_toilet', '/home/jcworkma/jack/3d-form/models/voxel_vaegan1/modelnet10/2019-03-15_17-08-43/tb')],
                #'tb_compare': [('best_vaegan', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-17_08-40-29/tb')],
                #'tb_compare': [('vaegan_100epochs_toilets', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-18_13-12-53/tb')],
                #'tb_compare': [('vaegan_logless_loss', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-22_16-01-54/tb')],
                'tb_compare': [('voxel_vae_toilet_200epochs_1', os.path.join(MODEL_DIR, 'voxel_vae_toilet_200epochs_1', 'tb'))],
                'no_gan': True,
                'monitor_memory': True,
                # these settings control how often the components' optimizers are executed during the training loop
                'train_vae_cadence': 1,
                'train_gan_cadence': 1,
                'dis_noise': 0,
                'adaptive_lr': False
            }
        }
    }
    
    return cfg

## Tensorboard Prep

We launch tensorboard with a call to the python subprocess module. Sometimes, that process does not die with the rest of the experiment and lingers on as a system process. This becomes a problem when we try to initialize tensorboard for the next experiment because they cannot share the same port!

The function below is designed to solve this problem. It uses the linux pgrep utility to search for existing tensorboard processes and kill them. Note that this probably won't work on Windows.

In [4]:
from utils import kill_tensorboard

kill_tensorboard()

['pgrep', 'tensorboard'] yielded -> b''


## Training

We start with a check that we are not attempting to overwrite the last MODEL_DIR. If you are blocked by the assert, re-execute the cfg code above to generate a new MODEL_DIR. This will allow you to move ahead with training.

The sacred experiment will save away a copy of your experiment settings in an experiments directory. This can be accessed later in case we need to retrieve a prime config.

If tensorboard is enabled, tune in at localhost:6006 or your_ip:6006
   

In [ ]:
cfg = make_cfg()
model_dir = cfg.get('cfg').get('model').get('ckpt_dir')
kill_tensorboard()

/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-24_15-16-59
['pgrep', 'tensorboard'] yielded -> b''


In [ ]:
if last_model_dir == model_dir:
    print('dont overwrite!')
    assert False
else:
    last_model_dir = model_dir

ex.run(config_updates=cfg)

WARNING - root - Added new config entry: "cfg.dataset.class"
WARNING - root - Added new config entry: "cfg.dataset.index"
WARNING - root - Added new config entry: "cfg.dataset.splits"
WARNING - root - Added new config entry: "cfg.generator.pad"
WARNING - root - Added new config entry: "cfg.generator.verbose"
WARNING - root - Added new config entry: "cfg.model.adaptive_lr"
WARNING - root - Added new config entry: "cfg.model.batch_size"
WARNING - root - Added new config entry: "cfg.model.ckpt_dir"
WARNING - root - Added new config entry: "cfg.model.debug"
WARNING - root - Added new config entry: "cfg.model.dec_lr"
WARNING - root - Added new config entry: "cfg.model.dec_weight"
WARNING - root - Added new config entry: "cfg.model.dev_step"
WARNING - root - Added new config entry: "cfg.model.dis_lr"
WARNING - root - Added new config entry: "cfg.model.dis_noise"
WARNING - root - Added new config entry: "cfg.model.display_step"
WARNING - root - Added new config entry: "cfg.model.enc_lr"
WARNI

Logging to /home/jcworkma/jack/3d-form/src/logs/2019-03-24_15-16__root.log
Starting train_vaegan main
Numpy random seed: 994715431
Saved cfg: /home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-24_15-16-59/cfg.json
Dataset: <class 'data.modelnet10.ModelNet10'>
Using dataset index /home/jcworkma/jack/3d-form/src/../data/processed/modelnet10_toilet_index.csv and pctile None
Shuffling dataset
dataset n_input=7104
Splitting Datasets
Num input = 7104
Num batches per epoch = 55.50
Initializing VoxelVaegan
encoder/enc_conv1/batchnorm/add_1:0: (?, 30, 30, 30, 8)
encoder/batch_normalization/batchnorm/add_1:0: (?, 15, 15, 15, 16)
encoder/batch_normalization_1/batchnorm/add_1:0: (?, 13, 13, 13, 32)
encoder/batch_normalization_2/batchnorm/add_1:0: (?, 7, 7, 7, 64)
encoder/batch_normalization_3/batchnorm/add_1:0: (?, 7, 7, 7, 343)
encoder/dense_1/BiasAdd:0: (?, 100)
encoder/dense_2/BiasAdd:0: (?, 100)
epsilon: (?, 100)
z: (?, 100)
input_z: (?, 100)
decoder/dec_dense1/BiasAdd:

<tf.Variable 'encoder/batch_normalization_2/moving_variance/Adam:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_2/moving_variance/Adam_1:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/dense/kernel/Adam:0' shape=(64, 343) dtype=float32_ref>
<tf.Variable 'encoder/dense/kernel/Adam_1:0' shape=(64, 343) dtype=float32_ref>
<tf.Variable 'encoder/dense/bias/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/dense/bias/Adam_1:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/gamma/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/gamma/Adam_1:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/beta/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/beta/Adam_1:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/moving_mean/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normaliz

<tf.Variable 'decoder/dec_conv4/kernel/Adam_1:0' shape=(4, 4, 4, 8, 16) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/gamma/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/gamma/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/beta/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/beta/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_mean/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_mean/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_variance/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_variance/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/dec_conv5/kernel/Adam:0' shape=(3, 3, 3, 1, 8) dtype=float32_ref>
<tf.Variable 'decoder/dec_conv5/kernel/Adam_1:0' shape=(3, 3, 3, 1, 8) dtype=fl

Memory Use (GB): 1.2758102416992188
Epoch: 0 / 201, Batch: 28 (0 / 3712), Elapsed time: 1.77 mins
Enc Loss = 997.84, KL Divergence = 0.01, Reconstruction Loss = 0.10, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 1.84 mins
Memory Use (GB): 1.20269775390625
Epoch: 0 / 201, Batch: 29 (0 / 3840), Elapsed time: 1.84 mins
Enc Loss = 944.39, KL Divergence = 0.01, Reconstruction Loss = 0.10, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 1.90 mins
Memory Use (GB): 1.1809349060058594
Epoch: 0 / 201, Batch: 30 (0 / 3968), Elapsed time: 1.90 mins
Enc Loss = 844.48, KL Divergence = 0.01, Reconstruction Loss = 0.09, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 1.96 mins
Memory Use (GB): 1.2553024291992188
Epoch: 0 / 201, Batch: 31 (0 / 4096), Elapsed time: 1.96 mins
Enc Loss = 872.48, KL Divergence = 0.02, Reconstruction Loss = 0.09, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 2.03 mins
Me

...

Enc Loss = 445.54, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 234.84 mins
Memory Use (GB): 1.1911201477050781
Epoch: 85 / 201, Batch: 36 (4736 / 5504), Elapsed time: 234.84 mins
Enc Loss = 449.96, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 234.90 mins
Memory Use (GB): 1.3368453979492188
Epoch: 85 / 201, Batch: 37 (4864 / 5504), Elapsed time: 234.90 mins
Enc Loss = 447.29, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 234.96 mins
Memory Use (GB): 1.1585807800292969
Epoch: 85 / 201, Batch: 38 (4992 / 5504), Elapsed time: 234.96 mins
Enc Loss = 438.87, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 235.03 mins
Memory Use (GB): 1.2531547546386719
Epoch: 85 / 201, Batch: 39 (5120 / 5

Memory Use (GB): 1.1800384521484375
Epoch: 86 / 201, Batch: 24 (3200 / 5504), Elapsed time: 236.79 mins
Enc Loss = 454.99, KL Divergence = 0.00, Reconstruction Loss = 0.05, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 236.85 mins
Memory Use (GB): 1.3372421264648438
Epoch: 86 / 201, Batch: 25 (3328 / 5504), Elapsed time: 236.86 mins
Enc Loss = 449.41, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 236.92 mins
Memory Use (GB): 1.3319740295410156
Epoch: 86 / 201, Batch: 26 (3456 / 5504), Elapsed time: 236.92 mins
Enc Loss = 441.30, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 236.98 mins
Memory Use (GB): 1.3246688842773438
Epoch: 86 / 201, Batch: 27 (3584 / 5504), Elapsed time: 236.98 mins
Enc Loss = 443.65, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -99

Epoch: 87 / 201, Batch: 12 (1664 / 5504), Elapsed time: 238.75 mins
Enc Loss = 452.80, KL Divergence = 0.00, Reconstruction Loss = 0.05, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 238.81 mins
Memory Use (GB): 1.3720245361328125
Epoch: 87 / 201, Batch: 13 (1792 / 5504), Elapsed time: 238.82 mins
Enc Loss = 437.27, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 238.88 mins
Memory Use (GB): 1.3469772338867188
Epoch: 87 / 201, Batch: 14 (1920 / 5504), Elapsed time: 238.88 mins
Enc Loss = 449.38, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 238.94 mins
Memory Use (GB): 1.3376274108886719
Epoch: 87 / 201, Batch: 15 (2048 / 5504), Elapsed time: 238.94 mins
Enc Loss = 437.65, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 239.01 mins
Memo

Enc Loss = 445.59, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 240.76 mins
Memory Use (GB): 1.2695465087890625
Epoch: 88 / 201, Batch: 1 (256 / 5504), Elapsed time: 240.76 mins
Enc Loss = 443.36, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 240.82 mins
Memory Use (GB): 1.1592025756835938
Epoch: 88 / 201, Batch: 2 (384 / 5504), Elapsed time: 240.82 mins
Enc Loss = 445.36, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 240.89 mins
Memory Use (GB): 1.2200660705566406
Epoch: 88 / 201, Batch: 3 (512 / 5504), Elapsed time: 240.89 mins
Enc Loss = 433.92, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 240.95 mins
Memory Use (GB): 1.1607131958007812
Epoch: 88 / 201, Batch: 4 (640 / 5504), El

Enc Loss = 441.29, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 242.77 mins
Memory Use (GB): 1.3584671020507812
Epoch: 88 / 201, Batch: 33 (4352 / 5504), Elapsed time: 242.77 mins
Enc Loss = 443.69, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 242.83 mins
Memory Use (GB): 1.3856697082519531
Epoch: 88 / 201, Batch: 34 (4480 / 5504), Elapsed time: 242.83 mins
Enc Loss = 449.10, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 242.90 mins
Memory Use (GB): 1.2964630126953125
Epoch: 88 / 201, Batch: 35 (4608 / 5504), Elapsed time: 242.90 mins
Enc Loss = 444.25, KL Divergence = 0.00, Reconstruction Loss = 0.04, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 242.96 mins
Memory Use (GB): 1.3221511840820312
Epoch: 88 / 201, Batch: 36 (4736 / 5